In [ ]:
import os
os.environ["PIP_CACHE_DIR"] = "/workspace/.cache/pip"
os.environ["HF_HOME"] = "/workspace/.cache/huggingface"

import sys
sys.path.append('.')
from src import (
    load_model_and_tokenizer, load_gsm8k_dataset,
    build_chat_input, generate_batch, save_results_jsonl,
    MODEL_NAME
)
from tqdm import tqdm
import json

In [2]:
!pip install --no-cache-dir -U "transformers>=4.51.0" accelerate datasets torch pandas tqdm nnsight

INFO: pip is looking at multiple versions of wsproto to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 110.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 171.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 229.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 206.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 337.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 168.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 184.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 508.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 175.8 MB/s eta 0:00:00a 0:00:

In [29]:
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install --no-cache-dir typing-extensions --upgrade


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!pip uninstall -y torchvision

Found existing installation: torchvision 0.19.1+cu124
Uninstalling torchvision-0.19.1+cu124:
  Successfully uninstalled torchvision-0.19.1+cu124


In [1]:
import typing_extensions, torch
# print("typing_extensions:", typing_extensions.__version__)
print("torch:", torch.__version__)


torch: 2.9.1+cu128


In [ ]:
train_problems = load_gsm8k_dataset(split="train", n_problems=500)


In [ ]:
model, tok, config = load_model_and_tokenizer(MODEL_NAME)

In [ ]:
def run_and_save_jsonl(
    problems,
    out_path="data/rollouts/gsm8k_rollouts.jsonl",
    n_problems=500,
    n_rollouts=10,
    batch_size=16,
    max_new_tokens=800,
    base_seed=42,
):
    """Generate rollouts for GSM8K problems."""
    problems = problems[:n_problems]
    records = []

    for r in range(n_rollouts):
        rollout_seed = base_seed + r

        for start in tqdm(range(0, n_problems, batch_size),
                          desc=f"rollout {r+1}/{n_rollouts}"):
            batch = problems[start:start+batch_size]
            q_list = [(p["question"], "baseline", None) for p in batch]

            prompts, gen_texts, _ = generate_batch(
                tok, model, q_list,
                max_new_tokens=max_new_tokens,
                enable_thinking=True,
                seed=rollout_seed,
            )

            for i, (p, prompt, gen) in enumerate(zip(batch, prompts, gen_texts)):
                records.append({
                    "problem_idx": p["idx"],
                    "rollout_idx": r,
                    "question": p["question"],
                    "answer": p["answer"],
                    "prompt": prompt,
                    "gen_text": gen,
                    "gen_cfg": {
                        "max_new_tokens": max_new_tokens,
                        "temperature": 0.6,
                        "top_p": 0.95,
                        "top_k": 20,
                        "seed": rollout_seed,
                    }
                })

    save_results_jsonl(records, out_path)


In [ ]:
run_and_save_jsonl(
    train_problems,
    out_path="gsm8k_rollouts.jsonl",
    n_problems=500,
    n_rollouts=10,
    batch_size=16,
    max_new_tokens=800,
    base_seed=42,
)


rollout 10/10: 100%|██████████| 7/7 [09:42<00:00, 83.23s/it]

Wrote: aqua_rollouts.jsonl


In [ ]:
from src import generate_one
prompt, gen_text, _ = generate_one(tok, model, train_problems[2]["question"], "baseline", None, max_new_tokens=2000)
print((prompt, gen_text))

('<|im_start|>system\nSolve the math question. Think step by step. At the end, output exactly one line: Answer: X where X is the numerical answer (an integer).<|im_end|>\n<|im_start|>user\nQuestion:\n{\'question\': \'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?\', \'answer\': "In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.\\nBetty\'s grandparents gave her 15 * 2 = $<<15*2=30>>30.\\nThis means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.\\n#### 5"}\n<|im_end|>\n<|im_start|>assistant\n', "<think>\nOkay, let's see. The problem is about Betty saving money for a wallet that costs $100. She has half of the money she needs. Her parents give her $15, and her grandparents give her twice as much as her parents. The question is how much more money she need